### Mass segregation

Analysis of the mass segregation in the stellar clusters using different indices <br>
In particular we use the MST-based metric (e.g. Yu et al. 2011)

In [1]:
using PyCall
using Distributions
using Statistics
using DataFrames
using Distances, LightGraphs, SimpleWeightedGraphs
using Random , Printf

rootdir = "/home/stephane/Science/GAIA"

push!(LOAD_PATH,"$rootdir/master/src")
using GaiaClustering

import PyPlot , CSV

@pyimport astroquery.vizier as vizier
@pyimport astropy.units as u
@pyimport astropy.coordinates as coord


## directory
wdir    = "$rootdir/products"
plotdir = "$rootdir/products/test"
ocdir= "$wdir/oc"

cd(wdir)

┌ Warning: `@pyimport foo` is deprecated in favor of `foo = pyimport("foo")`.
│   caller = _pywrap_pyimport(::PyObject) at PyCall.jl:407
└ @ PyCall /home/stephane/.julia/packages/PyCall/ttONZ/src/PyCall.jl:407


In [2]:
## Test OC
##

oc= CSV.read("$ocdir/NGC 6124-2.3deg-oc.csv" , delim= ";")

,ra,dec,l,b,distance,pmra,pmdec,X,Y,Z,vl,vb,vrad,gbar,rp,bp
,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰
1,248.314,-39.1181,342.912,5.9616,172.377,-11.8593,-22.0303,172.245,6.74447,0.122253,-19.8159,-5.02337,NaN,21.765,14.5745,17.9966
2,247.712,-42.5732,340.058,3.94601,175.615,-14.0032,-21.5762,175.501,-1.8729,-6.05217,-21.0746,-3.78274,NaN,23.5374,16.2922,20.0264
3,247.984,-42.4081,340.317,3.91201,182.366,-13.1387,-20.4854,182.251,-1.12312,-6.39299,-20.6992,-3.75543,NaN,22.9287,15.8706,19.4002
4,247.993,-42.1859,340.484,4.05878,176.536,-12.7136,-20.2254,176.441,-0.572013,-5.73663,-19.6369,-3.74155,NaN,20.1256,13.2496,15.6377
5,247.991,-42.1857,340.483,4.06,179.973,-12.207,-21.2668,179.877,-0.585808,-5.84449,-20.3749,-4.73645,NaN,19.6776,12.8861,15.1388
6,247.6,-38.7443,342.809,6.62348,178.495,-14.3495,-21.87,178.357,6.66125,2.1885,-21.8137,-3.73251,NaN,23.0481,15.9003,19.3954
7,246.336,-39.0719,341.888,7.11242,181.983,-12.0267,-20.1924,181.903,3.86828,3.78401,-19.7453,-4.59687,NaN,23.6782,16.5324,20.3177
8,247.668,-41.0375,341.16,5.01999,173.786,-14.0097,-21.1868,173.758,1.48651,-2.73268,-20.6282,-3.4996,NaN,21.8272,14.7329,17.8784
9,247.994,-41.2395,341.18,4.70271,172.12,-12.4389,-20.9532,172.075,1.5313,-3.65946,-19.4317,-4.19825,NaN,21.1651,13.8491,16.7635


In [3]:
## Create a graph with the selected stars
## ind: subset to select the x,y
## dtyp= "2d" or "3d"

function _mst_graph(ind,x,y,z=[] , dtyp="2d")
    nxy= length(ind)
    if dtyp == "2d"
        A= Array{Float64}(undef,2,nxy)
    elseif dtyp == "3d"
       A= Array{Float64}(undef,3,nxy) 
    end
    
    for i in ind
        if dtyp == "2d"
            A[:,i]= [x[i] y[i]]
        elseif dtyp == "3d"
            A[:,i]= [x[i] y[i] z[i]]
        end
    end
    
    d= Euclidean()
    p= pairwise(d, A, A, dims=2)
    println(p)
    
    src= Array{Int}(undef,0)
    dst=  Array{Int}(undef,0)
    wgt= Array{Float64}(undef,0)

    for i in 1:nxy
        for j in 1:nxy
            if i != j
                push!(src,i)
                push!(dst,j)
                push!(wgt,p[ind[i],ind[j]]) 
            end
        end
    end
    
    G= SimpleWeightedGraph(src, dst, wgt)
    
    kr= kruskal_mst(G, G.weights)
    
    for w in kr
        @printf("%d - %d : %3.3f\n",w.src,w.dst, w.weight)
    end
    
    return(kr)
end


_

ErrorException: syntax: all-underscore identifier used as rvalue around In[3]:49

In [4]:
## Main

_mst_graph(1:10, oc[:X], oc[:Y] , oc[:Z] , "2d")

[0.0 9.21178 12.7283 8.43434 10.5816 6.11212 10.0764 5.47114 5.21595 5.65202; 9.21178 0.0 6.79156 1.60535 4.56127 8.99939 8.59908 3.7847 4.82972 5.44968; 12.7283 6.79156 0.0 5.83546 2.43418 8.70397 5.00353 8.885 10.5166 7.08308; 8.43434 1.60535 5.83546 0.0 3.43527 7.48258 7.03848 3.38231 4.84684 4.00563; 10.5816 4.56127 2.43418 3.43527 0.0 7.4047 4.89317 6.46037 8.08407 5.0398; 6.11212 8.99939 8.70397 7.48258 7.4047 0.0 4.5136 6.92319 8.11058 3.59327; 10.0764 8.59908 5.00353 7.03848 4.89317 4.5136 0.0 8.48598 10.1019 5.10905; 5.47114 3.7847 8.885 3.38231 6.46037 6.92319 8.48598 0.0 1.68354 3.61419; 5.21595 4.82972 10.5166 4.84684 8.08407 8.11058 10.1019 1.68354 0.0 5.09995; 5.65202 5.44968 7.08308 4.00563 5.0398 3.59327 5.10905 3.61419 5.09995 0.0]
2 - 4 : 3.211
8 - 9 : 3.367
3 - 5 : 4.868
4 - 8 : 6.765
4 - 5 : 6.871
6 - 10 : 7.187
8 - 10 : 7.228
6 - 7 : 9.027
1 - 9 : 10.432


9-element Array{SimpleWeightedEdge{Int64,Float64},1}:
 Edge 2 => 4 with weight 3.2107043633996875
 Edge 8 => 9 with weight 3.3670824339553125
 Edge 3 => 5 with weight 4.868359710539991 
 Edge 4 => 8 with weight 6.764610098828963 
 Edge 4 => 5 with weight 6.870532292361459 
 Edge 6 => 10 with weight 7.186538923888171
 Edge 8 => 10 with weight 7.228380360452785
 Edge 6 => 7 with weight 9.027197540792072 
 Edge 1 => 9 with weight 10.431907380650877